In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q transformers>=4.49 torch evaluate scikit-learn pillow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [3]:
import os
import ast
import pandas as pd
from PIL import Image
import torch
from sklearn.model_selection import train_test_split  # scikit-learn split :contentReference[oaicite:2]{index=2}
import evaluate                                        # metrics :contentReference[oaicite:3]{index=3}

vqa_df = pd.read_csv('/kaggle/input/images-with-vqas/merged_image_data_vqa.csv')
vqa_df.head()


2025-05-12 10:41:19.073303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747046479.266141      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747046479.327947      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,image_id,image_path,listing,vqa_response
0,81iZlv3bjpL,Dataset/final_dataset/8ccb5859.jpg,"{""brand"": [{""language_tag"": ""nl_NL"", ""value"": ...","[[""What is the main color of the shoe?"", ""Brow..."
1,619y9YG9cnL,Dataset/final_dataset/9f76d27b.jpg,"{""item_dimensions"": {""height"": {""normalized_va...","[[""What color are the drawer slides?"", ""White""..."
2,81NP7qh2L6L,Dataset/final_dataset/665cc994.jpg,"{""item_dimensions"": {""height"": {""normalized_va...","[[""What color is the filament?"", ""Yellow""], [""..."
3,61Rp4qOih9L,Dataset/final_dataset/b4f9d0cc.jpg,"{""brand"": [{""language_tag"": ""en_GB"", ""value"": ...","[[""What is the primary color of the fabric?"", ..."
4,714CmIfKIYL,Dataset/final_dataset/2b1c2516.jpg,"{""brand"": [{""language_tag"": ""en_AU"", ""value"": ...","[[""What color is the shoe?"", ""Beige""], [""What ..."


In [4]:
print(vqa_df['vqa_response'].notna())

0        True
1        True
2        True
3        True
4        True
         ... 
19462    True
19463    True
19464    True
19465    True
19466    True
Name: vqa_response, Length: 19467, dtype: bool


In [5]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "ibm-granite/granite-vision-3.2-2b"
processor = AutoProcessor.from_pretrained(model_path, use_fast=True)
model = AutoModelForVision2Seq.from_pretrained(model_path).to(device)

processor_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
model.eval().to(device)

LlavaNextForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(729, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
          

In [7]:
from tqdm import tqdm
predictions, refs = [], []
count  = 0
start_time = time.time()
TIME_LIMIT = 42000

with torch.no_grad():
    for idx, row in tqdm(vqa_df.iterrows(), total=len(vqa_df)):
        elapsed_time = time.time()
        if elapsed_time - start_time > TIME_LIMIT:
            print(f"\nTime limit of {TIME_LIMIT} seconds exceeded. Aborting loop at index {idx}.")
            print(f'Number of successfully run images - {idx}')
            break
        image_path = os.path.join(
            "/kaggle/input/images-with-vqas/final_dataset/final_dataset",
            row["image_path"].replace("Dataset/final_dataset/", "")
        )
    
        try:
            image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            print(f"Warning: Image not found at {image_path}, skipping row {idx}")
            predictions.append("IMAGE_NOT_FOUND")
            refs.append(row["vqa_response"].strip() if pd.notna(row["vqa_response"]) else "")
            # count += 1
            # if count == 5:
            #     break
            continue
        except Exception as e:
            print(f"Error loading image {image_path}: {e}, skipping row {idx}")
            predictions.append("IMAGE_ERROR")
            refs.append(row["vqa_response"].strip() if pd.notna(row["vqa_response"]) else "")
            # count += 1
            # if count == 5:
            #     break
            continue
    
        try:
            vqa_pairs = ast.literal_eval(row["vqa_response"])
        except Exception as e:
            print(f"Failed to parse vqa_response: {e}, skipping row {idx}")
            predictions.append("PARSE_ERROR")
            refs.append(row["vqa_response"].strip() if pd.notna(row["vqa_response"]) else "")
            # count += 1
            # if count == 5:
            #     break
            continue
    
        for q, gt_answer in vqa_pairs:
            prompt = f"Based on the image, answer the following question with a single word. Question: {q} Answer:"
            conversation = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": prompt},
                    ],
                },
            ]
            # token = processor.tokenizer.image_token
            # print(token)
            # prompt = f"<image>\nBased on the image, answer the following question with a single word. Strictly give a single word as an answer, no sentences, just a single word. Your generated output should only be a single word which is the answer. If it's more than word, you will be penalized. Start the answer with the adjective. I want the start of the answer as the adjective otherwise you will be heavily penalized. Understand this properly for once, give the adjective describing the image according to the question as the first word of the answer and out only one word. Understand this otherwise you will be penalized heavily. Don't use short forms or abbreviations for the answer. Output the full word. Understand this carefully. The adjective should not be at the end of the answer. It should be at the start.  Question: {q} Answer:"
            # inputs = processor(text=prompt, image=image, return_tensors="pt")
    
            inputs = processor.apply_chat_template(
            conversation,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
            )
            inputs = {k: v.to("cuda") for k, v in inputs.items()}
            for k in inputs:
                if k != "input_ids" and k != "attention_mask":  # Keep input_ids and attention_mask as long/int
                    inputs[k] = inputs[k].to(torch.float16)
            output = model.generate(**inputs, max_new_tokens=10)
            pred_full = processor.decode(output[0], skip_special_tokens=True)
    
            if "Answer:" in pred_full:
                answer_portion = pred_full.split("Answer:\n<|assistant|>")[-1]
            else:
                answer_portion = pred_full
            words = answer_portion.split()
            if words:
                words = [t.rstrip('.,;:!?') for t in words if t.strip()]
                pred = "".join(words)
                # pred = words[0].rstrip('\'.,;:!?')
            else:
                pred = "" # Handle empty prediction
    
            # print(f"\nIdx: {idx}")
            # print(f"Original Question: {q}")
            # print(f"Prompt Used: '{prompt}'")
            # print(f"Full Prediction: '{pred_full}'")
            # print(f"Processed Prediction (Single Word Attempt): '{pred}'")
    
            count +=1
            predictions.append(pred)
            refs.append(gt_answer.strip() if pd.notna(gt_answer) else "")
            #print("count: ", count)
            # if (count == 100): # Limiting to 100 for the example
            #     print("\nReached count limit (100). Breaking loop.")
        #     break

print("length of predictions", len(predictions))

 51%|█████     | 9883/19467 [11:40:03<11:18:52,  4.25s/it]


Time limit of 42000 seconds exceeded. Aborting loop at index 9883.
Number of successfully run images - 9883
length of predictions 29525


In [8]:
!pip install git+https://github.com/google-research/bleurt.git
!pip install scikit-learn evaluate bert-score rouge-score sentence-transformers rapidfuzz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-hyf0dd78
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-hyf0dd78
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=bdd36946641afb65540637c73ffaf99cbc48d507cec3968064e8b708dcdef694
  Stored in directory: /tmp/pip-ephem-wheel-cache-g4o50yoi/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6aab140a0a6270df017e4cf3e3124d13c53d36f6d8c4df5cfa78aa9219b0e0f9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from bert_score import BERTScorer
from rouge_score import rouge_scorer
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# Normalize case
preds_l = [p.lower() for p in predictions]
refs_l = [r.lower() for r in refs]

# Compute exact-match binary metrics
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1] * len(refs)

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)

# Print metrics
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}")
print(f"Exact-match Recall:    {rec:.3f}")
print(f"Exact-match F1:        {f1:.3f}\n")

# Save predictions and ground truths to CSV
pred_ref_df = pd.DataFrame({
    "Prediction": predictions,
    "Ground_Truth": refs
})
pred_ref_df.to_csv('prediction_output.csv', index=False)

metrics_df = pd.DataFrame({
"Metric": ["Accuracy", "Precision", "Recall", "F1"],
"Value": [acc, prec, rec, f1]
})
metrics_df.to_csv('exact_metrics.csv',index=False)

Exact-match Accuracy: 0.674
Exact-match Precision: 1.000
Exact-match Recall:    0.674
Exact-match F1:        0.805



In [10]:
# Use second GPU
device = "cuda:1" if torch.cuda.device_count() > 1 else "cuda:0"

# Initialize result dictionary
final_metrics = {}

# --- ROUGE ---
print("\n--- ROUGE Scores ---")
rouge_eval_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores, rougeL_scores = [], []

for pred, ref in zip(preds_l, refs_l):
    if not pred or not ref:
        rouge1_scores.append(0.0)
        rougeL_scores.append(0.0)
        continue
    scores = rouge_eval_scorer.score(ref, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

if rouge1_scores:
    final_metrics["rouge1_f1"] = np.mean(rouge1_scores)
    final_metrics["rougeL_f1"] = np.mean(rougeL_scores)
    print(f"Average ROUGE-1 F1: {final_metrics['rouge1_f1']:.3f}")
    print(f"Average ROUGE-L F1: {final_metrics['rougeL_f1']:.3f}")
else:
    print("No ROUGE scores to compute.")

# --- BERTScore ---
print("\n--- BERTScore ---")
try:
    bert_eval_scorer = BERTScorer(lang="en", rescale_with_baseline=True, device=device)
    filtered_preds_l = [p for p, r in zip(preds_l, refs_l) if p and r]
    filtered_refs_l = [r for p, r in zip(preds_l, refs_l) if p and r]

    if filtered_preds_l and filtered_refs_l:
        P, R, F1 = bert_eval_scorer.score(filtered_preds_l, filtered_refs_l)
        final_metrics["bertscore_precision"] = P.mean().item()
        final_metrics["bertscore_recall"] = R.mean().item()
        final_metrics["bertscore_f1"] = F1.mean().item()
        print(f"Average BERTScore Precision: {final_metrics['bertscore_precision']:.3f}")
        print(f"Average BERTScore Recall:    {final_metrics['bertscore_recall']:.3f}")
        print(f"Average BERTScore F1:        {final_metrics['bertscore_f1']:.3f}")
    else:
        print("Not enough valid pairs for BERTScore.")
except Exception as e:
    print(f"Could not compute BERTScore: {e}")

# --- Levenshtein Normalized Similarity ---
print("\n--- Levenshtein Normalized Similarity ---")
lev_similarities = []
for pred, ref in zip(preds_l, refs_l):
    if not pred and not ref:
        similarity = 1.0
    elif not pred or not ref:
        similarity = 0.0
    else:
        similarity = Levenshtein.normalized_similarity(pred, ref)
    lev_similarities.append(similarity)

if lev_similarities:
    final_metrics["levenshtein_sim"] = np.mean(lev_similarities)
    print(f"Average Levenshtein Normalized Similarity: {final_metrics['levenshtein_sim']:.3f}")
else:
    print("No Levenshtein similarities to compute.")

# --- Sentence-BERT Cosine Similarity ---
print("\n--- Sentence-BERT Cosine Similarity ---")
try:
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    valid_pairs = [(p, r) for p, r in zip(preds_l, refs_l) if p and r]
    if valid_pairs:
        sbert_preds, sbert_refs = zip(*valid_pairs)
        embeddings_preds = sbert_model.encode(list(sbert_preds), convert_to_tensor=True)
        embeddings_refs = sbert_model.encode(list(sbert_refs), convert_to_tensor=True)
        cosine_scores = util.cos_sim(embeddings_preds, embeddings_refs)
        pairwise_sim = [cosine_scores[i, i].item() for i in range(len(valid_pairs))]
        final_metrics["sbert_cosine_sim"] = np.mean(pairwise_sim)
        print(f"Average Sentence-BERT Cosine Similarity: {final_metrics['sbert_cosine_sim']:.3f}")
    else:
        print("Not enough valid pairs for Sentence-BERT similarity.")
except Exception as e:
    print(f"Could not compute Sentence-BERT similarity: {e}")

# Save final metrics to CSV
df_metrics = pd.DataFrame([final_metrics])
df_metrics.to_csv("alternate_metrics.csv", index=False)

# Print summary
print("\n--- Final Metrics ---")
for k, v in final_metrics.items():
    print(f"{k}: {v:.3f}")

print("\nFinal metrics saved to 'metrics_output.csv'")


--- ROUGE Scores ---
Average ROUGE-1 F1: 0.687
Average ROUGE-L F1: 0.687

--- BERTScore ---


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/923 [00:00<?, ?it/s]

Batches:   0%|          | 0/923 [00:00<?, ?it/s]

Could not compute Sentence-BERT similarity: CUDA out of memory. Tried to allocate 3.25 GiB. GPU 0 has a total capacity of 15.89 GiB of which 3.14 GiB is free. Process 2577 has 12.75 GiB memory in use. Of the allocated memory 12.38 GiB is allocated by PyTorch, and 84.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

--- Final Metrics ---
rouge1_f1: 0.687
rougeL_f1: 0.687
bertscore_precision: 0.902
bertscore_recall: 0.915
bertscore_f1: 0.907
levenshtein_sim: 0.746

Final metrics saved to 'metrics_output.csv'
